题目: 给出一数列，在数列的柱状图里，找出最大的面积

基本思路(mono stack):
* 用mono stack来求解，复杂度O(n)
* 初始: height.append(0) 与 stack = [-1]
    * stack = [-1]是为了同时避免第一次空list调用和避免当pop后到头的情况，
        * stack = [0], pop后就空了，而w是从0到最右
    * height.append(0)则与⬆️配合。
* stack内是的index是单调递增的，index同时保存了顺序和边界。
* 在计算面积时，先pop后计算边界:
    * 每次计算矩阵，左边界一直是i
    * 先pop再计算width,是为了得到正确的右边界
    * 例如heights = [1, 6, 6], 那么最后stack=[-1, 0, 2], 此时如果先计算width是不行的

In [2]:
# 初解，超时
class Solution:
    def largestRectangleArea(self, heights):
        # search with order
        maxArea = 0
        m = len(heights)
        for width in range(m, 0, -1):
            # width is from m to 1
            maxAreaTemp = 0
            for start in range(m - width + 1):
                # from: m - width, ... ,0（也可写作range(m - width, -1, -1)）
                end = start + width
                minheight = min(heights[start: end])
                maxAreaTemp = max(maxAreaTemp, minheight * width)
            maxArea = max(maxArea, maxAreaTemp)
        return maxArea

In [ ]:
# 进一步优化...动态规划，仍然超时....(O(n^2))
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        # search with order
        maxArea = 0
        m = len(heights)
        minheights = [0] * m
        for width in range(1, m + 1):
            # width is from m to 1
            maxAreaTemp = 0
            for start in range(m - width + 1):
                end = start + width
                # 是否能对minheight 进行一些优化？
                # 用动态规划法最简单
                # 以下为进一步优化的动态规划法:
                if width == 1:
                    minheights[start] = heights[start]
                else:
                    minheights[start] = min(minheights[start], minheights[start+1])
                
                maxAreaTemp = max(maxAreaTemp, minheights[start] * width)
            maxArea = max(maxArea, maxAreaTemp)
        return maxArea

In [3]:
# 对于每一个柱子，你需要找到其右边第一个比他低的柱子，才可以一次性算出在其右边界前，包含自身的最大矩阵
# (在右边界之前，算出的矩阵面积，总会有更大的矩阵面积)
# 而"更新右边界"这个过程，可以用monoStack(单调栈)进行解决

class Solution:
    def largestRectangleArea(self, heights):
        heights.append(0)
        stack = [-1]
        ans = 0
        for i in range(len(height)):
            while heights[i] < heights[stack[-1]]:
                # 每次计算矩阵，左边界一直是i
                # 先pop再计算width,是为了得到正确的右边界
                # 例如heights = [1, 6, 6], 那么最后stack=[-1, 0, 2], 此时如果先计算width是不行的
                h = heights[stack.pop()]
                w = i - stack[-1] - 1
                ans = max(ans, h * w)
            stack.append(i)
        heights.pop()
        return ans

In [ ]:
# monoStack, by myself

class Solution:
    # 不考虑边缘情况的版本，对于最后一个柱子不好处理
    # 还存在许多问题，应该先pop再计算width
    def largestRectangleArea(self, height):
        stack = []
        ans = 0
        for i in range(len(height)):
            while stack and (height[stack[-1]] > height[i]):
                # increase inside the stack
                # i is the left border
                w = i - stack[-1]
                h = height[stack.pop()]
                ans = max(ans, w * h)
            stack.append(i)
        return ans